In [3]:
import lightkurve as lk
%matplotlib inline
toi = 573
tic = 296780789
search_result1 = lk.search_lightcurve('TIC'+str(tic), mission='TESS', exptime=20)
search_result2 = lk.search_lightcurve('TIC'+str(tic), mission='TESS', exptime=120)
print(search_result1)
print(search_result2)

SearchResult containing 1 data products.

 #     mission     year author exptime target_name distance
                                  s                 arcsec 
--- -------------- ---- ------ ------- ----------- --------
  0 TESS Sector 62 2023   SPOC      20   296780789      0.0
SearchResult containing 3 data products.

 #     mission     year author exptime target_name distance
                                  s                 arcsec 
--- -------------- ---- ------ ------- ----------- --------
  0 TESS Sector 08 2019   SPOC     120   296780789      0.0
  1 TESS Sector 35 2021   SPOC     120   296780789      0.0
  2 TESS Sector 62 2023   SPOC     120   296780789      0.0


In [30]:
lc_collection1 = search_result1[0].download()
lc_collection2 = search_result2[2].download()

lc1 = lc_collection1.normalize().remove_outliers(sigma_upper=3, sigma_lower=float('inf'))
lc2 = lc_collection2.normalize().remove_outliers(sigma_upper=3, sigma_lower=float('inf'))

import numpy as np
# Create array of periods to search
period = np.linspace(0.5, 20, 10000)
# Create a BLSPeriodogram

bls1 = lc1.to_periodogram(method='bls', period=period, frequency_factor=500);
bls2 = lc2.to_periodogram(method='bls', period=period, frequency_factor=500);


planet_b_period1 = bls1.period_at_max_power
planet_b_t01 = bls1.transit_time_at_max_power
planet_b_dur1 = bls1.duration_at_max_power
planet_b_depth1 = bls1.depth_at_max_power
planet_b_period2 = bls2.period_at_max_power
planet_b_t02 = bls2.transit_time_at_max_power
planet_b_dur2 = bls2.duration_at_max_power
planet_b_depth2 = bls2.depth_at_max_power

planet_b_mask1 = bls1.get_transit_mask(period=planet_b_period1,
                                     transit_time=planet_b_t01,
                                     duration=planet_b_dur1)
planet_b_mask2 = bls2.get_transit_mask(period=planet_b_period2,
                                     transit_time=planet_b_t02,
                                     duration=(planet_b_dur2+planet_b_dur2*0.12))

x = lc1.to_pandas()
y = lc2.to_pandas()
x_std = x['flux'].std()
y_std = y['flux'].std()

masked_lc1 = lc1[~planet_b_mask1]
masked_lc2 = lc2[~planet_b_mask2]

masked_x = masked_lc1.to_pandas()
maskedx_std = masked_x['flux'].std()

masked_y = masked_lc2.to_pandas()
maskedy_std = masked_y['flux'].std()


# lc_binned1=lc1.bin(time_bin_size=0.00139)
lc_binned1=lc1.bin(binsize=6)
binnedx = lc_binned1.to_pandas()
binnedx_std = binnedx['flux'].std()



bls3 = lc_binned1.to_periodogram(method='bls', period=period, frequency_factor=500);
planet_binned_period1 = bls3.period_at_max_power
planet_binned_t01 = bls3.transit_time_at_max_power
planet_binned_dur1 = bls3.duration_at_max_power
planet_binned_depth1 = bls3.depth_at_max_power


planet_b_mask_binned = bls3.get_transit_mask(period=planet_binned_period1,
                                     transit_time=planet_binned_t01,
                                     duration=planet_binned_dur1)

masked_binned_lc1 = lc_binned1[~planet_b_mask_binned]
masked_binned = masked_binned_lc1.to_pandas()
masked_binned_std = masked_binned['flux'].std()

# bls1 = lc_binned1.to_periodogram(method='bls', period=period, frequency_factor=500);

print(x_std,maskedx_std)
print(y_std,maskedy_std)
print(binnedx_std,masked_binned_std)


C:\Users\Hamish\AppData\Local\Temp\ipykernel_17508\461898692.py:48: LightkurveDeprecationWarning: "binsize" was deprecated in version 2.0 and will be removed in a future version. 
        Use time_bin_size instead.
  lc_binned1=lc1.bin(binsize=6)


0.014424563 0.012079027
0.009564497 0.0052509075
0.009678283587898214 0.005610571084827123
